Synthesize specifications from large experiments from Historia paper
=====================


In [1]:
val userhome = System.getProperty("user.home")
val jniPath = s"${userhome}/software/z3/build"
val expDirPath = "/Users/shawnmeier/Documents/source/historia/Historia/notebooks"
val newPath = Array(jniPath) ++  System.getProperty("java.library.path").split(":")
System.setProperty("java.library.path",newPath.distinct.mkString(":"))
val sysPathsField = classOf[ClassLoader].getDeclaredField("sys_paths");
sysPathsField.setAccessible(true);
sysPathsField.set(null, null);
val jarpath = s"${userhome}/Documents/source/historia/Historia/implementation/target/scala-2.13/soot_hopper-assembly-0.1.jar"
val datadir = "/Users/shawnmeier/Documents/data/historia_generalizability"
interp.load.cp(os.Path(jarpath))

import $ivy.`org.plotly-scala:plotly-almond_2.13:0.8.2`
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import $ivy.`com.github.nscala-time::nscala-time:2.32.0`

import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import plotly._, element._, layout._, Plotly._
import ujson.Value
import sys.process._

import $ivy.`com.lihaoyi::scalatags:0.12.0`

import jupyter.Displayer, jupyter.Displayers
import scala.collection.JavaConverters._
import scala.collection.mutable
import com.github.nscala_time.time.Imports._
import org.joda.time.Period

userhome: String = "/Users/shawnmeier"
jniPath: String = "/Users/shawnmeier/software/z3/build"
expDirPath: String = "/Users/shawnmeier/Documents/source/historia/Historia/notebooks"
newPath: Array[String] = Array(
  "/Users/shawnmeier/software/z3/build",
  "/Users/shawnmeier/Library/Java/Extensions",
  "/Library/Java/Extensions",
  "/Network/Library/Java/Extensions",
  "/System/Library/Java/Extensions",
  "/usr/lib/java",
  "."
)
res0_4: String = "/Users/shawnmeier/Library/Java/Extensions:/Library/Java/Extensions:/Network/Library/Java/Extensions:/System/Library/Java/Extensions:/usr/lib/java:."
sysPathsField: java.lang.reflect.Field = private static java.lang.String[] java.lang.ClassLoader.sys_paths
jarpath: String = "/Users/shawnmeier/Documents/source/historia/Historia/implementation/target/scala-2.13/soot_hopper-assembly-0.1.jar"
datadir: String = "/Users/shawnmeier/Documents/data/historia_generalizability"
import $ivy.$                                          

import $ivy.$         

In [ ]:
import better.files._

import scala.util.Random
import edu.colorado.plv.bounder.{Driver,ExpTag,Action,RunConfig,PickleSpec}
import edu.colorado.plv.bounder.lifestate.{SpecSpace,ViewSpec,SpecSignatures}
import edu.colorado.plv.bounder.symbolicexecutor.state.{InitialQuery,Reachable,ReceiverNonNull, DisallowedCallin}
import upickle.default.read
import upickle.default.write
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}
import edu.colorado.plv.bounder.ExperimentsDb
import edu.colorado.plv.bounder.BounderUtil
//import scala.concurrent.duration._
import scala.language.postfixOps
import slick.driver.H2Driver.api._
import slick.jdbc.GetResult
import slick.jdbc.SQLActionBuilder
import scala.concurrent.Await
import almond.interpreter.api.DisplayData
import edu.colorado.plv.bounder.ir.Messages


Driver.setZ3Path(s"${userhome}/software/z3/build")

// var android_home_possible = List(s"${userhome}/Library/Android/sdk", s"${userhome}/Android/Sdk")
// var android_home = android_home_possible.find(p => File(p).exists()).get             
// BounderUtil.setEnv(Map("DYLD_LIBRARY_PATH" -> s"${userhome}/software/z3/build","ANDROID_HOME" -> android_home,"HOME" -> userhome))

System.setProperty("user.dir", s"${System.getProperty("user.home")}/Documents/source/bounder/notebooks/ossExp/SpecGen");
val expDir = File(expDirPath)

In [ ]:
println(s"Samples to specify: ${expDir.glob("**/config.json").toList.size}")

In [ ]:
case class Count(n:Int)
case class StrRes(s:String)
implicit val getCountResult = GetResult(r => Count(r.<<))
implicit val getStrResult = GetResult(r => StrRes(r.<<))

//implement in a class so any sequence gets converted to a table that can be displayed in jupyter
trait TableAble {
  def headers:List[String]   
  def values:List[Any]
}
case class Table(v:Seq[TableAble]){

}
Displayers.register(classOf[Table], (t: Table) => {
  import scalatags.Text.all._
  Map(
    "text/html" -> {
      table(cls:="table")(
        tr(t.v.head.headers.map(v => th(v))),
        for (row <- t.v) yield tr(
          row.values.map{
              case v:String => td(v)
              case v:Int => td(v)
          }
        )
      ).render
    }
  ).asJava
})

In [ ]:

import edu.colorado.plv.bounder.lifestate.LifeState.{AbsMsg, LSSpec, LSPred, OAbsMsg, SetSignatureMatcher, Signature, SignatureMatcher, SubClassMatcher,Or,And,NS,Not}
import edu.colorado.plv.bounder.symbolicexecutor.state.{BoolVal, Equals, NamedPureVar, NotEquals, NullVal, PureExpr, TopVal}


import edu.colorado.plv.bounder.ir.{CBEnter, CBExit, CIExit}

object ToSpecify{
    println(expDir)
    def allCfg = (expDir / "SpecGen").glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
    val preRunCfg = (expDir / "data").glob("**/config.json").map{f => (f,read[RunConfig](f.contentAsString))}.toList
    
}
val f = NamedPureVar("f")
val b = NamedPureVar("b")
val l = NamedPureVar("l")
val a = NamedPureVar("a")
val p = NamedPureVar("p")
println(ToSpecify.allCfg.size)

case class AppCitation(name:String, date:DateTime, url:String, pkg:String){
    
}
object AppCitation{
    val citations = List(
        AppCitation("Seafile",(new DateTime).withYear(2022).withMonthOfYear(8).withDayOfMonth(23), 
                    "https://f-droid.org/en/packages/com.seafile.seadroid2/", "com.seafile.seadroid2"),
        AppCitation("BatteryBot", (new DateTime).withYear(2021).withMonthOfYear(12).withDayOfMonth(07),
            "https://f-droid.org/en/packages/com.darshancomputing.BatteryIndicatorPro/","com.darshancomputing.BatteryIndicatorPro"),
        AppCitation("Vanilla Music", (new DateTime).withYear(2023).withMonthOfYear(2).withDayOfMonth(18), 
                    "https://f-droid.org/en/packages/ch.blinkenlights.android.vanilla/", "ch.blinkenlights.android.vanilla"),
        AppCitation("Syncthing", (new DateTime).withYear(2023).withMonthOfYear(01).withDayOfMonth(08),
            "https://f-droid.org/en/packages/com.nutomic.syncthingandroid/","com.nutomic.syncthingandroid"),
        AppCitation("OpenVPN", (new DateTime).withYear(2022).withMonthOfYear(12).withDayOfMonth(17),
                    "https://f-droid.org/en/packages/de.blinkt.openvpn/" ,"de.blinkt.openvpn"),
        AppCitation("Navit", (new DateTime).withYear(2021).withMonthOfYear(03).withDayOfMonth(11),
            "https://f-droid.org/en/packages/org.navitproject.navit/","org.navitproject.navit"),
        AppCitation("Connectbot", (new DateTime).withYear(2022).withMonthOfYear(8).withDayOfMonth(23),
                    "https://github.com/connectbot/connectbot", "org.connectbot"),

        AppCitation("Antennapod", (new DateTime).withYear(2023).withMonthOfYear(1).withDayOfMonth(1),
                    "https://github.com/AntennaPod/AntennaPod", "de.danoeh.antennapod")
    )
    def fromFolder(folder:String):AppCitation = {
        println(folder)
        citations.find(c => folder.contains(c.pkg)).getOrElse(throw new Exception(s"citation not found for folder: ${folder}"))
    }
}
object Location{
    def apply(i:InitialQuery):String = {
        val fullClass = i match{
            case i:ReceiverNonNull => i.sig.base
            case i:DisallowedCallin => i.className
            case _ => ???
        }
        val spl = fullClass.split('.')
        if(spl.isEmpty){
            throw new RuntimeException(s"could not parse initial query: $i")
        }
        
        val className = spl.last
        className.split('$').head
    }
}

trait Specifyable[T<:InitialQuery] extends TableAble{
    def initial:T
    def configWithSpec:RunConfig
    def reachableDual:InitialQuery
    lazy val mConfig = ToSpecify.allCfg.filter{case (_,cfg) => cfg.initialQuery.exists(v => v == initial)}
    lazy val configFile = mConfig.head._1
    def getCitation = {
        AppCitation.fromFolder(configFile.toString)
    }
    lazy val getMsg = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec" // change to messages_all.json for full non-filtered
        val toRead = msgFolder.glob("**/messages_component.json").toList
        if(toRead.size == 1){
            val msgFile = toRead.toList.head
            val out = read[Messages](msgFile.contentAsString)
            out
        }else {
             Messages(-1, -1, -1, -1,-1, -1,-1, -1)   
        }
    }
    lazy val getAllMsg = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec" // change to messages_all.json for full non-filtered
        val toRead = msgFolder.glob("**/messages_all.json").toList
        if(toRead.size == 1){
            val msgFile = toRead.toList.head
            val out = read[Messages](msgFile.contentAsString)
            out
        }else {
             Messages(-1, -1, -1, -1,-1, -1,-1, -1)   
        }  
    }
    lazy val getRes = {
        val rootDir = configFile.parent
        val msgFolder = rootDir / "spec"
        val resFile = msgFolder.glob("**/result_0.txt").toList
        if(resFile.size == 1){
            ujson.read(resFile.head.contentAsString)
        }else ujson.read("{}")
    }
    def getConfigWithSpecPath:File = configFile.parent / "config_spec.json"
    def runHistoriaWithSpec():Driver.LocResult = {
        val javaMemLimit=8 // Gb Note that this only limits JVM not JNI which can go significantly higher
        val historiaJar = jarpath
        val apkRootDir = datadir
        val outDir = getConfigWithSpecPath.parent.toString
        println(s"out directory: ${outDir}")
        val config = read[RunConfig](getConfigWithSpecPath.contentAsString)
        println(s"apk path: ${config.apkPath}")
        val outSubdir = config.outFolder.get.replace("${baseDirOut}",outDir)
        println(s"out subdirectory: ${config.outFolder}")
        val cmd = s"java -Xmx${javaMemLimit}G -jar ${historiaJar} -m verify -c ${getConfigWithSpecPath} -b ${apkRootDir} -u ${outDir} -o MEM --debug"
        BounderUtil.runCmdStdout(cmd)

        // val infoCmd = cmd.replace("verify","info") // uncomment to recompute message info for table 3
        // val outDb = outSubdir / "out.db"
        // if(outDb.exists){
        //     outDb.delete()
        // }
        
        // BounderUtil.runCmdStdout(infoCmd)

        // println((outSubdir / "messages_component.json").contentAsString)

        read[Driver.LocResult]((outSubdir / "result_0.txt").contentAsString)
    }
    def writeSpec():Unit = {
        // verify with added specification
        val outf = getConfigWithSpecPath
        outf.overwrite(write(configWithSpec))
        
        //check that line is reachable under normal circumstances
        val outfReach = configFile.parent / "config_spec_locreach.json"
        outfReach.overwrite(write(configWithSpec.copy(initialQuery = List(reachableDual))))
    }
    def result:String
    def specificationTime:Duration
    def readableSpecificationTime:String = {
        scala.math.BigDecimal(specificationTime.minutes).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toInt.toString
    }
  lazy val appT = getCitation.name
  lazy val appCbMsgT = getMsg.cbSize
   
  lazy val texTag = configWithSpec.tag.heuristicType match{
      case "SensitiveDerefCallinCaused" => "\\apGa"
      case "Disallow.I_CIEnter_AsyncTaskexecute" => "\\apEx"
      case "Disallow.I_CIEnter_Dialogdismiss" => "\\ymDi"
      case "SensitiveDerefFieldCausedFinish" => "\\cbFi"
      case "SensitiveDerefFieldCausedSynch" => "\\synch"
      case v => 
         throw new RuntimeException(s"did not find tex version of tag: ${v}")
  }
  lazy val columns = List(
      ("app", getCitation.name),
      ("location", Location(initial)),
      ("pattern", texTag),
      ("cb/cbret", getAllMsg.cbSize),
      ("ci", getAllMsg.syntCi),
      ("filt cb/cbret", getMsg.cbSize),
      ("filt ci", getMsg.syntCi),
      ("specs", configWithSpec.specSet.getSpecSet.size + (initial match{
          case _:ReceiverNonNull => 0
          case _:DisallowedCallin => 1
          case _ => throw new RuntimeException(s"unsupported initial: ${initial}")
      })),
      ("cb", getMsg.matchedCb),
      ("cb\\%", scala.math.BigDecimal(getMsg.matchedCb.toDouble / getMsg.cbSize.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("cbret", getMsg.matchedCbRet),
      ("cbret\\%", scala.math.BigDecimal(getMsg.matchedCbRet.toDouble / getMsg.cbSize.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("ci", getMsg.matchedSyntCi),
      ("ci\\%", scala.math.BigDecimal(getMsg.matchedSyntCi.toDouble / getMsg.syntCi.toDouble * 100).setScale(0,scala.math.BigDecimal.RoundingMode.HALF_UP).toString),
      ("time", if(result == "timeout") "" else getRes("time").num.toInt),
      //("depth", if(result == "timeout") "" else getRes("depthChar")("cbDepth").num.toInt),
      ("res", result),
      ("spec time", readableSpecificationTime)
      ).unzip
  def headers:List[String] = columns._1
  def values:List[Any]     = columns._2
}

In [ ]:
import edu.colorado.plv.bounder.lifestate.FragmentGetActivityNullSpec
case object Specify_3 extends Specifyable[ReceiverNonNull]{ // 4:55 + :55
    //ossExp/SpecGen/com.seafile.seadroid2/SensitiveDerefCallinCaused/0
    val initial = read[InitialQuery]("""
    {
      "methodName": "android.app.Dialog onCreateDialog(android.os.Bundle)",
      "line": 68,
      "t": "ReceiverNonNull",
      "matcher": "android.view.LayoutInflater getLayoutInflater()",
      "className": "com.seafile.seadroid2.ui.dialog.SslConfirmDialog"
    }
    """).asInstanceOf[ReceiverNonNull]
    val reachableDual = Reachable(initial.sig, initial.line)
    assert(mConfig.size == 1, s"wrong config file count: ${mConfig.size}")
    val config = mConfig.head._2
    val DialogFragment = Set("android.support.v4.app.DialogFragment")
    val Fragment_onCreateDialog = AbsMsg(CBEnter, 
                                         SubClassMatcher(DialogFragment,"android.app.Dialog onCreateDialog\\(android.os.Bundle\\)", "Fragment_onCreateDialog"), 
                                         TopVal::f::Nil)
    val Fragment_onCreate: SignatureMatcher = SubClassMatcher(SpecSignatures.Fragment,
          "void onCreate\\(android.os.Bundle\\)", "Fragment_onCreate")
                                                                                                             
    val Fragment_onCreate_entry: OAbsMsg = AbsMsg(CBEnter, Fragment_onCreate, TopVal::f::Nil)
    val Fragment_resumed = And(
        NS(Fragment_onCreate_entry, SpecSignatures.Fragment_onDestroy_exit),
        NS(SpecSignatures.Fragment_onActivityCreated_entry, SpecSignatures.Fragment_onDestroy_exit))
    val DialogFragment_onCreateDialog_whenCreated = LSSpec(f::Nil, Nil, Fragment_resumed, 
                                                           Fragment_onCreateDialog)
        val fragmentActivityNotAttached_improved: LSPred =
        Or(And(
            NS(SpecSignatures.Fragment_onDestroy_exit, SpecSignatures.Fragment_onActivityCreated_entry),
            NS(SpecSignatures.Fragment_onDestroy_exit, Fragment_onCreate_entry)
        ),
          And(Not(SpecSignatures.Fragment_onActivityCreated_entry), Not(Fragment_onCreate_entry)))
    val FragmentGetActivityNullSpec_improved = FragmentGetActivityNullSpec.getActivityNull.copy(pred = fragmentActivityNotAttached_improved)
    lazy val configWithSpec = config.copy(specSet = PickleSpec(specs = 
                                                          Set(DialogFragment_onCreateDialog_whenCreated,  FragmentGetActivityNullSpec_improved),
                                                          disallow=Set()))
    val manualNoSpecRun = "alarm"
    val result = "proven with spec" //checked 3/31/23
    val specificationTime = (4.minutes + 55.seconds + 
                             55.seconds).toDuration
}
Specify_3.writeSpec()
Specify_3.configFile
println(s"App Name: ${Specify_3.appT}")

//Uncomment the following two lines to run Historia on this sample from Table 3
val result = Specify_3.runHistoriaWithSpec()
println(s"Result Summary: ${result.resultSummary}")
